In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain_core.runnables import RunnableSequence
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [16]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [1]:
import os

In [2]:
questions = [
    "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
    "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
    "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
    "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
    "Quais são os principais aspectos da crítica social e política presentes em \"Os Sertões\"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?",
]

In [7]:
#Load dos modelos (Embeddings e LLM)

embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name= "gpt-3.5-turbo", max_tokens = 200)

In [8]:
#Carregar o PDF
pdf_path = os.path.join("..", "data", "os-sertoes.pdf")
loader = PyPDFLoader(pdf_path, extract_images=False)
pages = loader.load_and_split()

In [9]:
#Separar em chunks (pedaços de documento)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 800,
    chunk_overlap = 150,
    length_function = len,
    add_start_index = True,
    separators=["\n\n", "\n", ".", "!", "?", ",", " "]
)
chunks = text_splitter.split_documents(pages)

In [11]:
#Salvar no vectorDB

db = Chroma.from_documents(chunks, embedding=embeddings_model)

In [12]:
#Load Retriever
retriever = db.as_retriever(search_kwargs={"k": 3})
#Construção da cadeia de prompt para a chamada do LLM
chain = load_qa_chain(llm, chain_type="stuff")

/var/folders/pp/byh0kn390x12x8dgh89r2d8r0000gn/T/ipykernel_42731/2150703760.py:4: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="stuff")


In [13]:
TEMPLATE = """
Você é um especialista em literatura brasileira. Responda a pergunta abaixo utilizando o contexto informado

Contexto: {context}
    
Pergunta: {question}
"""

prompt = PromptTemplate(input_variables=["context", "question"], template=TEMPLATE)
parser = StrOutputParser()
sequence = RunnableSequence(prompt | llm | parser)

In [14]:
def answer_question(question: str):
    context = retriever.invoke(question)

    response = sequence.invoke({"context": context, "question": question})

    return response

In [ ]:
for index, question in enumerate(questions):
  
    resposta = answer_question(question)
    
    print({"numero": index, "pergunta": question, "resposta": resposta})

{'numero': 0, 'pergunta': 'Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?', 'resposta': 'Euclides da Cunha descreve o ambiente natural do sertão nordestino como sendo caracterizado por características geológicas e topográficas distintas. Ele destaca a influência dos elementos físicos do local, como as formações geológicas e o clima, na vida dos habitantes da região. O autor ressalta a importância desses fatores na configuração do ambiente e na forma como eles afetam a vida das pessoas que ali habitam.'}
{'numero': 1, 'pergunta': 'Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?', 'resposta': 'Euclides da Cunha descreve a população sertaneja como dispersa na multidão, mas enérgica e definida quando resumida em uma individualidade. Ele menciona que todas as crenças ingênuas, desde o fetichismo 